In [5]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [6]:
#### access URL in browser
url = 'https://www.spareroom.co.uk/flatshare/?search_id=1323266289&mode=list'

browser = webdriver.Chrome()
browser.get(url)